In [2]:
import os
import cv2
import glob
import torch
import random
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from scipy.io import loadmat
from diffusers import MarigoldDepthPipeline
from transformers import AutoImageProcessor, AutoModelForDepthEstimation, ZoeDepthForDepthEstimation

2026-01-09 15:40:34.201605: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767973234.399527      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767973234.456964      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767973234.907353      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767973234.907395      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767973234.907398      55 computation_placer.cc:177] computation placer alr

In [3]:
IBIMS_PATH = "/kaggle/input/ibims-1/iBims-1"
NYU_PATH = "/kaggle/input/nyu-depth-v2/nyu_data/data/nyu2_test"

DEVICE = torch.device("cuda") if torch.cuda.is_available else torch.device("cpu")
print(DEVICE)

cuda


In [4]:
# https://www.asg.ed.tum.de/lmf/ibims1/

class IBimsLoader:
    def __init__(self, root_dir=IBIMS_PATH):
        self.rgb_files = sorted(glob.glob(os.path.join(root_dir, "rgb", "*.png")))
        self.depth_files = sorted(glob.glob(os.path.join(root_dir, "ibims1_core_mat", "*.mat")))
        
        if len(self.rgb_files) != len(self.depth_files):
            print("Hmm something is wrong with the dataset...")

    def __len__(self):
        return len(self.rgb_files)

    def get_item(self, idx):
        img_path = self.rgb_files[idx]
        img = cv2.imread(img_path)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        depth_path = self.depth_files[idx]
        mat_data = loadmat(depth_path)

        gt_depth = mat_data['data']['depth'][0][0]

        mask = gt_depth > 0.001
        
        return img, gt_depth, mask,img_path

class NyuLoader:
    def __init__(self, root_dir=NYU_PATH, samples_size=100, seed=42):
        self.samples = []
        
        self.rgb_files = sorted(glob.glob(os.path.join(root_dir, "*_colors.png"), recursive=True))
        print(f"Found {len(self.rgb_files)} RGB candidates. Matching with Depth...")

        for rgb_path in self.rgb_files:
            depth_path = rgb_path.replace("_colors.png", "_depth.png")
            if os.path.exists(depth_path):
                self.samples.append((rgb_path, depth_path))

        if samples_size is not None and len(self.samples) > samples_size:
            random.seed(seed)
            self.samples = random.sample(self.samples, samples_size)
    
    def __len__(self):
        return len(self.samples)

    def get_item(self, idx):
        img_path, depth_path = self.samples[idx]
        
        img = cv2.imread(img_path)
        depth_png = cv2.imread(depth_path, -1)
        if depth_png is None:
            raise ValueError(f"Failed to load depth: {depth_path}")

        gt_depth = depth_png.astype(np.float32) / 1000.0
        mask = gt_depth > 0.001

        return img, gt_depth, mask, img_path

In [8]:
#  https://huggingface.co/blog/Isayoften/monocular-depth-estimation-guide

def get_absrel(gt, pred):
    return np.mean(np.abs(gt-pred)/gt)

def get_delta(gt, pred, exponent=1):
    inlier = np.maximum((gt/pred), (pred/gt))
    return np.mean(inlier < 1.25**exponent)

def get_silog(gt, pred):
    """
    Computes Scale Invariant Logarithmic Error.
    Lower is better.
    """
    pred = np.maximum(pred, 1e-5)
    gt = np.maximum(gt, 1e-5)

    err = np.log(pred) - np.log(gt)

    silog = np.sqrt(np.mean(err ** 2) - (np.mean(err)) ** 2)
    
    return silog * 100

def align_depth_least_square(gt_arr, pred_arr, valid_mask_arr):
    ori_shape = pred_arr.shape

    gt = gt_arr.squeeze()  # [H, W]
    pred = pred_arr.squeeze()
    valid_mask = valid_mask_arr.squeeze()
    gt_masked = gt[valid_mask].reshape((-1, 1))
    pred_masked = pred[valid_mask].reshape((-1, 1))

    # numpy solver
    _ones = np.ones_like(pred_masked)
    A = np.concatenate([pred_masked, _ones], axis=-1)
    X = np.linalg.lstsq(A, gt_masked, rcond=None)[0]
    scale, shift = X

    aligned_pred = pred_arr * scale + shift

    # restore dimensions
    aligned_pred = aligned_pred.reshape(ori_shape)

    return aligned_pred, scale, shift

In [6]:
class ModelWrapper:
    def __init__(self, choice):
        self.choice = choice
        
        if "marigold" in choice.lower():
            self.model = MarigoldDepthPipeline.from_pretrained(choice, variant="fp16").to(DEVICE)
            self.model.set_progress_bar_config(disable=True)
            print("found marigold model")
        elif "zoedepth" in choice.lower():
            self.processor = AutoImageProcessor.from_pretrained(choice)
            self.model = ZoeDepthForDepthEstimation.from_pretrained(choice).to(DEVICE)
            print("found zoedepth model")
        else:
            self.processor = AutoImageProcessor.from_pretrained(choice)
            self.model = AutoModelForDepthEstimation.from_pretrained(choice).to(DEVICE)

    def infer(self, image_path, marigold_steps=4):
        if "marigold" in self.choice.lower():
            image = Image.open(image_path).convert("RGB")
            w0, h0 = image.size
            
            with torch.no_grad():
                pipe_out = self.model(
                    image, 
                    num_inference_steps=marigold_steps, 
                    output_type="pt"
                )
            prediction = pipe_out.prediction

            # Check if resize is needed
            if prediction.shape[-2:] != (h0, w0):
                prediction = torch.nn.functional.interpolate(
                    prediction,
                    size=(h0, w0),
                    mode="bicubic",
                    align_corners=False
                )

            return prediction.squeeze().cpu().numpy()

        # Transformer-based models inference
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        inputs = self.processor(images=image, return_tensors="pt").to(DEVICE)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            predicted_depth = outputs.predicted_depth
        
        # Resize to original image size
        prediction = torch.nn.functional.interpolate(
            predicted_depth.unsqueeze(1),
            size=image.shape[:2],
            mode="bicubic",
            align_corners=False,
        )
        
        return prediction.squeeze().cpu().numpy()

In [75]:
# Check if dataset gt is in mm
# dataset = ...
for i in range(len(dataset)):
    _, gt, _, _ = dataset.get_item(i)
    if np.median(gt) > 100:
        print("maybe that measurement is in millimeters... needs to change!")

In [16]:
ibims_loader = IBimsLoader()
nyu_loader = NyuLoader()

class ModelConfig:
    def __init__(self, model_choice, display_name, relative=True, ibims=ibims_loader, nyu=nyu_loader):
        self.model = ModelWrapper(model_choice)
        self.display_name = display_name
        self.relative = relative
        self.ibims = ibims
        self.nyu = nyu
        self.absrel = {}
        self.d1 = {}
        self.silog = {}

    def set_results(self, dataset, absrel, d1, silog):
        self.absrel[dataset] = absrel;
        self.d1[dataset] = d1
        self.silog[dataset] = silog

    def get_absrel_result(self, dataset):
        return self.absrel[dataset]

    def get_d1_result(self, dataset):
        return self.d1[dataset]

    def get_silog_result(self, dataset):
        return self.silog[dataset]

    def __basic_benchmark(self, dataset, debug=True):
        absrel_list = []
        delta_list = []
        silog_list = []
        
        ds = "ibims"
        if isinstance(dataset, NyuLoader):
            ds = "nyu"
            
        for i in tqdm(range(len(dataset))):
            _, gt, mask, image_path = dataset.get_item(i)
            prediction = self.model.infer(image_path)

            if self.relative:
                depth, _, _ = align_depth_least_square(gt, prediction, mask)
                absrel = get_absrel(gt[mask], depth[mask])
                delta = get_delta(gt[mask], depth[mask], 1)
                silog = get_silog(gt[mask], depth[mask])
            else:
                absrel = get_absrel(gt[mask], prediction[mask])
                delta = get_delta(gt[mask], prediction[mask], 1)
                silog = get_silog(gt[mask], prediction[mask])

            absrel_list.append(absrel)
            delta_list.append(delta)
            silog_list.append(silog)
            
        am = np.mean(absrel_list)
        dm = np.mean(delta_list)
        sm = np.mean(silog_list)
        self.set_results(ds, am, dm, sm)

        if debug:
            print(f"\tAverage AbsRel = {am:.4f}")
            print(f"\tAverage Delta1 = {dm:.4f}")
            print(f"\tAverage SILog  = {sm:.4f}")

    def benchmark(self, debug=True):
        print(f"{'='*20}[ {self.display_name} ]{'='*20}")
        if self.ibims is not None:
            print("IBims-1 dataset:")
            self.__basic_benchmark(self.ibims, debug=debug)
        if self.nyu is not None:
            print("NYU Depth V2 dataset:")
            self.__basic_benchmark(self.nyu, debug=debug)
        print(f"{'='*20}[ {self.display_name} ]{'='*20}\n")

Found 654 RGB candidates. Matching with Depth...


In [17]:
configs = [
    ModelConfig("depth-anything/Depth-Anything-V2-Large-hf", "DAV2-Large"),
    ModelConfig("depth-anything/Depth-Anything-V2-Metric-Indoor-Large-hf", "DAV2-Indoor-Metric", relative=False)
]

for config in configs:
    config.benchmark(debug=True)

# TO-DO: Search Eigen-Crop technique to fix the results!

====================[ DAV2-Large ]====================
IBims-1 dataset:


100%|██████████| 100/100 [00:41<00:00,  2.43it/s]


	Average AbsRel = 0.1253
	Average Delta1 = 0.8572
	Average SILog  = 51.3739
NYU Depth V2 dataset:


100%|██████████| 100/100 [00:38<00:00,  2.60it/s]


	Average AbsRel = 0.1659
	Average Delta1 = 0.7953
	Average SILog  = 83.3935
====================[ DAV2-Large ]====================

====================[ DAV2-Indoor-Metric ]====================
IBims-1 dataset:


100%|██████████| 100/100 [00:37<00:00,  2.68it/s]


	Average AbsRel = 0.1265
	Average Delta1 = 0.8861
	Average SILog  = 7.5893
NYU Depth V2 dataset:


100%|██████████| 100/100 [00:35<00:00,  2.84it/s]

	Average AbsRel = 0.2250
	Average Delta1 = 0.6607
	Average SILog  = 20.1663
====================[ DAV2-Indoor-Metric ]====================



In [18]:
configs[0].get_absrel_result("ibims")

np.float64(0.1252612876460376)

In [22]:
name_map = {
    "DAV2-Large": "depth-anything/Depth-Anything-V2-Large-hf",
    "DAV2-Indoor-Metric": "depth-anything/Depth-Anything-V2-Metric-Indoor-Large-hf",
    "ZoeDepth": "Intel/zoedepth-nyu-kitti",
    "MiDaS-3.0": "Intel/dpt-large",
    "MiDaS-3.1": "Intel/dpt-beit-large-512",
    "Marigold-1.1": "prs-eth/marigold-depth-v1-1"
}

# model : [display_name, relative, ibims, nyu]
configurations = {
    "depth-anything/Depth-Anything-V2-Large-hf": ["DAV2-Large", True, True, True],
    "depth-anything/Depth-Anything-V2-Metric-Indoor-Large-hf": ["DAV2-Indoor-Metric", False, True, True],
    "Intel/zoedepth-nyu-kitti": ["ZoeDepth", False, True, True],
    "Intel/dpt-large": ["MiDaS-3.0", True, True, True], # MiDaS 3.0
    "Intel/dpt-beit-large-512": [True, True, True], # MiDaS 3.1
    "prs-eth/marigold-depth-v1-1": [True, True, True]
}

results = {}

for model in configurations.keys():
    config = configurations[model]
    relative = config[0]
    print(f"{'='*20}[ {model} ]{'='*20}\n")
    if config[1]:
        print("IBims-1 dataset:")
        benchmark(ModelWrapper(model), ibims, relative=relative)
    if config[2]:
        print("NYU Depth V2 dataset:")
        benchmark(ModelWrapper(model), nyu, relative=relative)
    print(f"{'='*20}[ {model} ]{'='*20}\n")

====================[ depth-anything/Depth-Anything-V2-Large-hf ]====================

IBims-1 dataset:


100%|██████████| 100/100 [00:38<00:00,  2.61it/s]


Average Abs Rel: 0.1252612876460376
Average d_1: 0.8572297850934841
NYU Depth V2 dataset:


100%|██████████| 100/100 [00:36<00:00,  2.74it/s]


Average Abs Rel: 0.1659458577632904
Average d_1: 0.7952931966145833
====================[ depth-anything/Depth-Anything-V2-Large-hf ]====================

====================[ depth-anything/Depth-Anything-V2-Metric-Indoor-Large-hf ]====================

IBims-1 dataset:


...

Average Abs Rel: 0.05358334528379531
Average d_1: 0.9676189089772491
NYU Depth V2 dataset:


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

found marigold model


100%|██████████| 100/100 [02:38<00:00,  1.59s/it]

Average Abs Rel: 0.09601327776908875
Average d_1: 0.8955147786458332
====================[ prs-eth/marigold-depth-v1-1 ]====================

